In [1]:
from samap.mapping import SAMAP
from samap.analysis import (get_mapping_scores, GenePairFinder, transfer_annotations,
                             sankey_plot, chord_plot, CellTypeTriangles, 
                             ParalogSubstitutions, FunctionalEnrichment,
                             convert_eggnog_to_homologs, GeneTriangles)
from samalg import SAM
import pandas as pd

Tutorial modified from https://github.com/atarashansky/SAMap/blob/main/SAMap_vignette.ipynb

# Running BLAST

SAMap requires BLAST results between all pairs of species; in this case, the salamander Pleurodeles waltl (`pw`), the turtle Trachemys scripta / Chrysemys picta (`cp`), and the lizard Pogona vitticeps (`pv`).

BLAST was run with a custom bash script `map_genes.sh` written by A. Tarashansky https://github.com/atarashansky/SAMap; the script auto-generates a directory called `maps/` in which the results will be deposited. The precomputed BLAST outputs are provided in the folder `SAMap_maps/`.

Reciprocal BLAST hits will be used to construct the gene homology graph used by SAMap.

# Running SAMap

SAMap accepts file paths to unprocessed, raw `.h5ad` files; these files were obtained from Seurat objects using the convertFormat function from the R package sceasy (from="Seurat", to="anndata").  

## Loading in raw data

In [5]:
fn3 = '/burg/tosches/users/mt3353/SAMap/datasets/Cp_neurons_3sp_integration_final.h5ad'
fn1 = '/burg/tosches/users/mt3353/SAMap/datasets/Pw_neurons_3sp_integration_final.h5ad'
fn2 = '/burg/tosches/users/mt3353/SAMap/datasets/Pv_neurons_3sp_integration_final.h5ad'

SAMap expects the above to be in a dictionary keyed by the species IDs determined in the BLAST step:

In [6]:
filenames = {'cp': fn3, 'pw':fn1,'pv':fn2}

SAMAP objects instantiated below. 

In [ ]:
sm = SAMAP(
        filenames,
        f_maps = '/burg/tosches/users/mt3353/SAMap/processed/',
        save_processed=True #if False, do not save the processed results to `*_pr.h5ad`
   )

In [2]:
# Start here if the SAM objects have been created and saved already

fn3 = '/burg/tosches/users/mt3353/SAMap/datasets/Cp_neurons_3sp_integration_final_pr.h5ad'
fn1 = '/burg/tosches/users/mt3353/SAMap/datasets/Pw_neurons_3sp_integration_final_pr.h5ad'
fn2 = '/burg/tosches/users/mt3353/SAMap/datasets/Pv_neurons_3sp_integration_final_pr.h5ad'

sam1=SAM()
sam1.load_data(fn1)

sam2=SAM()
sam2.load_data(fn2)

sam3=SAM()
sam3.load_data(fn3)

Finally, rather than determining neighborhood sizes by hopping along each cell's outgoing edges, we can also determine them using the cell type labels. In other words, cells that have the same label all belong to the same neighborhood. This can be set using the `neigh_from_keys` parameter of `SAMAP.run`.

The SAM files inherited metadata from the original Seurat objects. "tosches_annot", "cluster_label" and "unassigned" are the metadata columns for the cluster names of the lizard, salamander, and turtle data, respectively. 

In [ ]:
sams = {'pw':sam1,'pv':sam2,'cp':sam3}
sm = SAMAP(
        sams,
        f_maps = '/burg/tosches/users/mt3353/SAMap/processed/',
        keys = {'pv':'tosches_annot','pw':'cluster_label', 'cp':'unassigned'},
    )
sm.run(neigh_from_keys = {'pv':True,'pw':True, 'cp':True})

# samap.utils.save_samap(sm, 'pv_pw.sam')

In [17]:
# to save SAMap objects 
from samap.utils import save_samap

save_samap(sm, '/burg/tosches/users/mt3353/SAMap/datasets/3sp_integration_granular_220613_SAM.pkl')

In [13]:
# to import objects that were already saved
from samap.utils import load_samap

fn = '/burg/tosches/users/mt3353/SAMap/datasets/salamander_lizard_integration_220607_SAM.pkl'

sm = SAM()
load_samap(fn)

# Post-SAMap analysis


To calculate alignment scores between cell types, we can use `get_mapping_scores`. This function will use the combined SAM object produced by SAMap to calculate alignment scores between cell types in the provided cell type annotation columns of `sam.adata.obs`. 

The resulting tables show the highest-scoring alignment scores for each cell type (`D`) and pairwise mapping scores between cell types (`MappingTable`).

In [23]:
keys = {'pv':'tosches_annot','pw':'cluster_label','cp':'unassigned'}
D,MappingTable = get_mapping_scores(sm,keys,n_top = 10)

In [24]:
D.head()

pw_TEGLU30                   pv_PThE_2                   pw_TEGABA24  \
     Cluster Alignment score     Cluster Alignment score       Cluster   
0  pv_PThE_2        0.998491  pw_TEGLU30        0.998491  pv_MGE_INs_3   
1     cp_e17        0.153284  pw_TEGLU35        0.755401        cp_i12   
2     cp_e10         0.05515  pw_TEGLU39        0.656101  pv_MGE_INs_1   
3     cp_e02        0.028806      cp_e17        0.613492  pv_MGE_INs_2   
4     cp_e11        0.018458  pw_TEGLU41        0.525004        cp_i09   

                  pv_MGE_INs_3                      pv_Chol                  \
  Alignment score      Cluster Alignment score      Cluster Alignment score   
0        0.998261  pw_TEGABA24        0.998261  pw_TEGABA32        0.997721   
1        0.979571  pw_TEGABA25        0.876744  pw_TEGABA37        0.287141   
2        0.886297       cp_i09        0.866054       cp_e17        0.128316   
3        0.777871       cp_i08        0.814856  pw_TEGABA36        0.075507   
4        0.750855       cp_i10        0.652454       cp_e13        0.061634   

   ...   pw_TEGABA53                   pw_TEGABA19                 pw_TEGLU36  \
   ...       Cluster Alignment score       Cluster Alignment score    Cluster   
0  ...       pv_OT_1        0.151136  pv_CGE_INs_1        0.142309  pv_PThE_2   
1  ...  pv_CGE_INs_1        0.150791         pv_34        0.109761     cp_e02   
2  ...        cp_e17        0.149877       pv_OT_1         0.08901     cp_e17   
3  ...  pv_MGE_INs_3        0.104033        cp_e17        0.072664     cp_e19   
4  ...     pv_PThE_2        0.077746  pv_MGE_INs_1        0.056158     cp_e11   

                     pw_TEGABA29                 pw_TEGLU31                  
  Alignment score        Cluster Alignment score    Cluster Alignment score  
0        0.140198   pv_CGE_INs_1        0.126669  pv_PThE_2        0.063613  
1          0.0518         cp_i15        0.072909     cp_e17        0.018194  
2        0.049424         cp_i16        0.031955     cp_e02         0.01402  
3        0.023714   pv_MGE_INs_3        0.028531     cp_e10        0.008528  
4        0.016587  pv_Str_PENK_2        0.022047  pv_Sept_2        0.002218  

[5 rows x 350 columns]

In [13]:
MappingTable.head()

,pw_DIMEGABA1,pw_DIMEGABA2,pw_DIMEGLU1,pw_DIMEGLU2,pw_DIMEGLU3,pw_DIMEGLU4,pw_DIMEGLU5,pw_DIMEGLU6,pw_TEGABA1,pw_TEGABA10,...,pv_Sept_1,pv_Sept_2,pv_Str_PENK_1,pv_Str_PENK_2,pv_Str_TAC1,pv_aDVR_1,pv_aDVR_2,pv_aDVR_3,pv_aDVR_4,pv_pDVR
pw_DIMEGABA1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.008628,0.0,0.054077,0.012336,0.001682,0.000903,0.000000,0.000000,0.122586,0.009047
pw_DIMEGABA2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.046626,0.0,0.001794,0.081573,0.006341,0.004030,0.002638,0.000000,0.067009,0.000000
pw_DIMEGLU1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.036991
pw_DIMEGLU2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.006736,0.0,0.003224,0.061044,0.046855,0.231890,0.002063,0.005057,0.207696,0.038520
pw_DIMEGLU3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.013825,0.023879,0.000000,0.000000,0.000000,0.000000,0.000000


In [9]:
# save mapping table
keys = {'pv':'tosches_annot','pw':'cluster_label','cp':'superclusters'}
D,MappingTable = get_mapping_scores(sm,keys,n_top = 0)
#cropping mapping table because it contains everything twice:
mt = D
mt.to_csv('/burg/tosches/users/mt3353/SAMap/datasets/pw_pv_cp_220613_SAM_table.csv')

In [21]:
# saving UMAP coordinates computed by SAMap
sams = sm.sams
pv_umap_coord = sams['pv'].adata.obsm['X_umap_samap']
pw_umap_coord = sams['pw'].adata.obsm['X_umap_samap']
cp_umap_coord = sams['cp'].adata.obsm['X_umap_samap']
pv_cell_names = sams['pv'].adata.obs_names
pw_cell_names = sams['pw'].adata.obs_names
cp_cell_names = sams['cp'].adata.obs_names
pv_umap = pd.DataFrame(pv_umap_coord, index=pv_cell_names, columns=['UMAP_1','UMAP_2'])
pw_umap = pd.DataFrame(pw_umap_coord, index=pw_cell_names, columns=['UMAP_1','UMAP_2'])
cp_umap = pd.DataFrame(cp_umap_coord, index=cp_cell_names, columns=['UMAP_1','UMAP_2'])

pv_umap.to_csv('/burg/tosches/users/mt3353/SAMap/datasets/3sp_SAMap_UMAP_pv.csv')
pw_umap.to_csv('/burg/tosches/users/mt3353/SAMap/datasets/3sp_SAMap_UMAP_pw.csv')
cp_umap.to_csv('/burg/tosches/users/mt3353/SAMap/datasets/3sp_SAMap_UMAP_cp.csv')